## Merge data

In [3]:
import polars as pl
import gc
from sklearn.metrics import average_precision_score

In [2]:
outrider_input_path = "/s/project/abexp_veff/paper_submission/zenodo_user_update1/aberrant_expression_prediction_benchmark.parquet/**/*.parquet"
# old model
feature_old_input_path = "/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/models/fset@DNA_only_splice_nosv/data.parquet/**/*.parquet"
AbExp_old_input_path = "/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/predict_folds/subset=all/fset=DNA_only_splice_nosv/gtex_v8_old_dna/dna_only/DNA_only_splice_nosv@train_simplecv.py#lightgbm/data.parquet/**/*.parquet"
# new model
feature_new_input_path = "/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/models/fset@DNA_only_splice_Enformer_Aparent2_SpeciesLM_nosv/data.parquet/**/*.parquet"
AbExp_new_input_path = "/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/predict_folds/subset=all/fset=DNA_only_splice_Enformer_Aparent2_SpeciesLM_nosv/gtex_v8_old_dna/dna_only/DNA_only_splice_Enformer_Aparent2_SpeciesLM_nosv@train_simplecv.py#lightgbm/data.parquet/**/*.parquet"

In [3]:
outrider_input_path = "/s/project/abexp_veff/paper_submission/zenodo_user_update1/aberrant_expression_prediction_benchmark.parquet"

In [5]:
outrider_input_df = (pl.scan_parquet(outrider_input_path).rename({'tissue': 'subtissue', 'tissue_type': 'tissue'})
                     .filter(pl.col('is_in_benchmark'))
                     .select(['individual', 'subtissue', 'gene', 'tissue']))
outrider_input_df.head().collect()

individual,subtissue,gene,tissue
str,str,str,str
"""GTEX-1117F""","""Adipose - Subcutaneous""","""ENSG00000000003""","""Adipose Tissue"""
"""GTEX-1117F""","""Adipose - Subcutaneous""","""ENSG00000000419""","""Adipose Tissue"""
"""GTEX-1117F""","""Adipose - Subcutaneous""","""ENSG00000000457""","""Adipose Tissue"""
"""GTEX-1117F""","""Adipose - Subcutaneous""","""ENSG00000000938""","""Adipose Tissue"""
"""GTEX-1117F""","""Adipose - Subcutaneous""","""ENSG00000000971""","""Adipose Tissue"""


In [6]:
old_feature_df = pl.scan_parquet(feature_old_input_path)
old_feature_columns = [c for c in old_feature_df.columns if c.startswith('feature.')]
old_feature_df = old_feature_df.select(['individual', 'gene', 'subtissue', 'zscore', 'hilo_padj', *old_feature_columns])
old_abexp_df = pl.scan_parquet(AbExp_old_input_path).select(['individual', 'gene', 'subtissue', 'y_pred'])
old_df = old_feature_df.join(old_abexp_df, on=['individual', 'gene', 'subtissue'], how='inner')
old_df.head().collect()

/scratch/tmp/tsi/ipykernel_2295311/1641837241.py:2: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  old_feature_columns = [c for c in old_feature_df.columns if c.startswith('feature.')]


individual,gene,subtissue,zscore,hilo_padj,feature.expected_expr@theta,feature.splice_ensemble@AbSplice,feature.splice_ensemble@MMSplice_SpliceMap_Psi_ref,feature.splice_ensemble@SpliceAI,feature.vep@features.cadd_raw.max,feature.vep@features.LoF_HC.proportion,feature.vep@features.transcript_ablation.proportion,feature.vep@features.stop_gained.proportion,feature.vep@features.frameshift_variant.proportion,feature.vep@features.coding_sequence_variant.proportion,feature.vep@features.missense_variant.proportion,feature.vep@features.inframe_deletion.proportion,feature.vep@features.inframe_insertion.proportion,feature.vep@features.stop_lost.proportion,feature.vep@features.3_prime_UTR_variant.proportion,feature.vep@features.5_prime_UTR_variant.proportion,feature.vep@features.NMD_transcript_variant.proportion,feature.vep@features.NMD_escaping_variant.proportion,feature.vep@features.start_lost.proportion,feature.vep@features.splice_donor_variant.proportion,feature.vep@features.splice_acceptor_variant.proportion,feature.vep@features.splice_region_variant.proportion,y_pred
str,str,str,f32,i64,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64
"""GTEX-117XS""","""ENSG00000010270""","""Artery - Aorta""",-0.379722,0,425.092072,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.003991
"""GTEX-117XS""","""ENSG00000050130""","""Artery - Aorta""",-0.854305,0,311.207581,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.003991
"""GTEX-117XS""","""ENSG00000071282""","""Artery - Aorta""",1.144592,0,35.510849,0.000006,null,0.0,-0.110699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002247
"""GTEX-117XS""","""ENSG00000072609""","""Artery - Aorta""",-0.149068,0,455.533722,0.000006,null,0.01,0.786077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000538
"""GTEX-117XS""","""ENSG00000091039""","""Artery - Aorta""",-0.085687,0,464.469269,0.000006,null,0.0,0.018342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004298


In [7]:
old_df = old_df.join(outrider_input_df, on=['individual', 'subtissue', 'gene'], how='inner')

In [8]:
old_df = old_df.collect()

In [9]:
old_df.head()

individual,gene,subtissue,zscore,hilo_padj,feature.expected_expr@theta,feature.splice_ensemble@AbSplice,feature.splice_ensemble@MMSplice_SpliceMap_Psi_ref,feature.splice_ensemble@SpliceAI,feature.vep@features.cadd_raw.max,feature.vep@features.LoF_HC.proportion,feature.vep@features.transcript_ablation.proportion,feature.vep@features.stop_gained.proportion,feature.vep@features.frameshift_variant.proportion,feature.vep@features.coding_sequence_variant.proportion,feature.vep@features.missense_variant.proportion,feature.vep@features.inframe_deletion.proportion,feature.vep@features.inframe_insertion.proportion,feature.vep@features.stop_lost.proportion,feature.vep@features.3_prime_UTR_variant.proportion,feature.vep@features.5_prime_UTR_variant.proportion,feature.vep@features.NMD_transcript_variant.proportion,feature.vep@features.NMD_escaping_variant.proportion,feature.vep@features.start_lost.proportion,feature.vep@features.splice_donor_variant.proportion,feature.vep@features.splice_acceptor_variant.proportion,feature.vep@features.splice_region_variant.proportion,y_pred,tissue
str,str,str,f32,i64,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,str
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Subcutaneous""",0.323383,0,78.591827,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.00246,"""Adipose Tissue"""
"""GTEX-1117F""","""ENSG00000000419""","""Adipose - Subcutaneous""",-0.649543,0,612.874451,null,null,null,-0.065188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003511,"""Adipose Tissue"""
"""GTEX-1117F""","""ENSG00000000457""","""Adipose - Subcutaneous""",-0.260915,0,374.369202,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.003511,"""Adipose Tissue"""
"""GTEX-1117F""","""ENSG00000000938""","""Adipose - Subcutaneous""",2.05477,0,47.495094,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.00246,"""Adipose Tissue"""
"""GTEX-1117F""","""ENSG00000000971""","""Adipose - Subcutaneous""",-0.722197,0,25.436121,null,null,null,0.1505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00246,"""Adipose Tissue"""


In [10]:
old_df.shape

(99459917, 29)

In [11]:
old_benchmark_path = '/s/project/rep/processed/training_results_v16/DNA_only_splice_nosv_with_benchmark.parquet'

In [12]:
old_df.write_parquet(old_benchmark_path)

In [13]:
del old_df
gc.collect()

In [14]:
new_feature_df = pl.scan_parquet(feature_new_input_path)
new_feature_columns = [c for c in new_feature_df.columns if c.startswith('feature.')]
new_feature_df = new_feature_df.select(['individual', 'gene', 'subtissue', 'zscore', 'hilo_padj', *new_feature_columns])
new_abexp_df = pl.scan_parquet(AbExp_new_input_path).select(['individual', 'gene', 'subtissue', 'y_pred'])
new_df = new_feature_df.join(new_abexp_df, on=['individual', 'gene', 'subtissue'], how='inner')
new_df.head().collect()

/scratch/tmp/tsi/ipykernel_2295311/11030580.py:2: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  new_feature_columns = [c for c in new_feature_df.columns if c.startswith('feature.')]


individual,gene,subtissue,zscore,hilo_padj,feature.aparent2@Aparent2,feature.enformer@Enformer,feature.specieslm@pas_score,feature.specieslm@tss_score,feature.splice_ensemble@AbSplice,feature.splice_ensemble@MMSplice_SpliceMap_Psi_ref,feature.splice_ensemble@SpliceAI,feature.vep@features.cadd_raw.max,feature.vep@features.LoF_HC.proportion,feature.vep@features.transcript_ablation.proportion,feature.vep@features.stop_gained.proportion,feature.vep@features.frameshift_variant.proportion,feature.vep@features.coding_sequence_variant.proportion,feature.vep@features.missense_variant.proportion,feature.vep@features.inframe_deletion.proportion,feature.vep@features.inframe_insertion.proportion,feature.vep@features.stop_lost.proportion,feature.vep@features.3_prime_UTR_variant.proportion,feature.vep@features.5_prime_UTR_variant.proportion,feature.vep@features.NMD_transcript_variant.proportion,feature.vep@features.NMD_escaping_variant.proportion,feature.vep@features.start_lost.proportion,feature.vep@features.splice_donor_variant.proportion,feature.vep@features.splice_acceptor_variant.proportion,feature.vep@features.splice_region_variant.proportion,feature.expected_expr@theta,y_pred
str,str,str,f32,i64,f64,f32,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64
"""GTEX-117XS""","""ENSG00000010270""","""Artery - Aorta""",-0.379722,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,425.092072,0.003799
"""GTEX-117XS""","""ENSG00000050130""","""Artery - Aorta""",-0.854305,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,311.207581,0.003799
"""GTEX-117XS""","""ENSG00000071282""","""Artery - Aorta""",1.144592,0,null,null,-0.041191,null,0.000006,null,0.0,-0.110699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.510849,0.002456
"""GTEX-117XS""","""ENSG00000072609""","""Artery - Aorta""",-0.149068,0,null,null,null,null,0.000006,null,0.01,0.786077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,455.533722,0.002453
"""GTEX-117XS""","""ENSG00000091039""","""Artery - Aorta""",-0.085687,0,null,null,-0.040564,-0.030068,0.000006,null,0.0,0.018342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464.469269,0.003493


In [15]:
new_df = new_df.join(outrider_input_df, on=['individual', 'subtissue', 'gene'], how='inner')

In [16]:
new_df = new_df.collect()

In [17]:
new_df.shape

(99459917, 33)

In [18]:
new_df.head()

individual,gene,subtissue,zscore,hilo_padj,feature.aparent2@Aparent2,feature.enformer@Enformer,feature.specieslm@pas_score,feature.specieslm@tss_score,feature.splice_ensemble@AbSplice,feature.splice_ensemble@MMSplice_SpliceMap_Psi_ref,feature.splice_ensemble@SpliceAI,feature.vep@features.cadd_raw.max,feature.vep@features.LoF_HC.proportion,feature.vep@features.transcript_ablation.proportion,feature.vep@features.stop_gained.proportion,feature.vep@features.frameshift_variant.proportion,feature.vep@features.coding_sequence_variant.proportion,feature.vep@features.missense_variant.proportion,feature.vep@features.inframe_deletion.proportion,feature.vep@features.inframe_insertion.proportion,feature.vep@features.stop_lost.proportion,feature.vep@features.3_prime_UTR_variant.proportion,feature.vep@features.5_prime_UTR_variant.proportion,feature.vep@features.NMD_transcript_variant.proportion,feature.vep@features.NMD_escaping_variant.proportion,feature.vep@features.start_lost.proportion,feature.vep@features.splice_donor_variant.proportion,feature.vep@features.splice_acceptor_variant.proportion,feature.vep@features.splice_region_variant.proportion,feature.expected_expr@theta,y_pred,tissue
str,str,str,f32,i64,f64,f32,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,str
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Subcutaneous""",0.323383,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,78.591827,0.00251,"""Adipose Tissue"""
"""GTEX-1117F""","""ENSG00000000419""","""Adipose - Subcutaneous""",-0.649543,0,null,null,-0.038405,null,null,null,null,-0.065188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,612.874451,0.003407,"""Adipose Tissue"""
"""GTEX-1117F""","""ENSG00000000457""","""Adipose - Subcutaneous""",-0.260915,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,374.369202,0.003407,"""Adipose Tissue"""
"""GTEX-1117F""","""ENSG00000000938""","""Adipose - Subcutaneous""",2.05477,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,47.495094,0.00251,"""Adipose Tissue"""
"""GTEX-1117F""","""ENSG00000000971""","""Adipose - Subcutaneous""",-0.722197,0,null,null,-0.034286,null,null,null,null,0.1505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.436121,0.00251,"""Adipose Tissue"""


In [21]:
new_benchmark_path = '/s/project/rep/processed/training_results_v16/Enformer_Aparent2_SpeciesLM_with_benchmark.parquet'

In [22]:
new_df.write_parquet(new_benchmark_path)

In [23]:
del new_df
gc.collect()

## Read benchmarks

In [1]:
import polars as pl
import gc
from sklearn.metrics import average_precision_score

In [2]:
old_benchmark_path = '/s/project/rep/processed/training_results_v16/DNA_only_splice_nosv_with_benchmark.parquet'
new_benchmark_path = '/s/project/rep/processed/training_results_v16/Enformer_Aparent2_SpeciesLM_with_benchmark.parquet'
specieslm_raw_path = '/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/raw/SpeciesLM/*.parquet'

In [3]:
old_df = pl.scan_parquet(old_benchmark_path).select(['individual', 'gene', 'subtissue', 'tissue', 'hilo_padj', 'y_pred'])
old_df = old_df.with_columns(y_true=pl.col("hilo_padj") == -1,
                    y_pred=-pl.col('y_pred'))
old_df.head().collect()

individual,gene,subtissue,tissue,hilo_padj,y_pred,y_true
str,str,str,str,i64,f64,bool
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false
"""GTEX-1117F""","""ENSG00000000419""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003511,false
"""GTEX-1117F""","""ENSG00000000457""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003511,false
"""GTEX-1117F""","""ENSG00000000938""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false
"""GTEX-1117F""","""ENSG00000000971""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false


In [4]:
old_tissue_auprc = old_df.collect().group_by('tissue').map_groups(lambda group: pl.DataFrame({
    "tissue" : group["tissue"][0],
    "AUPRC" : average_precision_score(group.select('y_true').to_series(), group.select('y_pred').to_series())
}))

In [5]:
old_tissue_auprc.select(pl.col('AUPRC').median())

AUPRC
f64
0.058331


In [6]:
average_precision_score(old_df.select('y_true').collect().to_series(), old_df.select('y_pred').collect().to_series())

np.float64(0.06860093482034606)

In [7]:
new_df = (pl.scan_parquet(new_benchmark_path)
          .select(['individual', 'gene', 'subtissue', 'tissue', 'hilo_padj', 'y_pred', 'feature.specieslm@pas_score', 'feature.specieslm@tss_score',])
          .rename({'feature.specieslm@tss_score': 'tss_score',
                   'feature.specieslm@pas_score': 'pas_score'})
          .with_columns(tss_score=pl.col('tss_score').cast(pl.String),
                        pas_score=pl.col('pas_score').cast(pl.String)))
new_df = new_df.with_columns(y_true=pl.col("hilo_padj") == -1,
                    y_pred=-pl.col('y_pred'))
new_df.head().collect()

individual,gene,subtissue,tissue,hilo_padj,y_pred,pas_score,tss_score,y_true
str,str,str,str,i64,f64,str,str,bool
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00251,null,null,false
"""GTEX-1117F""","""ENSG00000000419""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003407,"""-0.03840475""",null,false
"""GTEX-1117F""","""ENSG00000000457""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003407,null,null,false
"""GTEX-1117F""","""ENSG00000000938""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00251,null,null,false
"""GTEX-1117F""","""ENSG00000000971""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00251,"""-0.034286473""",null,false


In [8]:
new_tissue_auprc = new_df.collect().group_by('tissue').map_groups(lambda group: pl.DataFrame({
    "tissue" : group["tissue"][0],
    "AUPRC" : average_precision_score(group.select('y_true').to_series(), group.select('y_pred').to_series())
}))

In [9]:
new_tissue_auprc.select(pl.col('AUPRC').median())

AUPRC
f64
0.067075


In [10]:
average_precision_score(new_df.select('y_true').collect().to_series(), new_df.select('y_pred').collect().to_series())

np.float64(0.07639739169256077)

In [11]:
merged_df= old_df.join(new_df, on=['individual', 'gene', 'subtissue', 'tissue', 'hilo_padj', 'y_true'], how='left', suffix='_new').rename({'y_pred': 'y_pred_old'})

In [12]:
merged_df.head().collect()

individual,gene,subtissue,tissue,hilo_padj,y_pred_old,y_true,y_pred_new,pas_score,tss_score
str,str,str,str,i64,f64,bool,f64,str,str
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,null,null
"""GTEX-1117F""","""ENSG00000000419""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003511,false,-0.003407,"""-0.03840475""",null
"""GTEX-1117F""","""ENSG00000000457""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003511,false,-0.003407,null,null
"""GTEX-1117F""","""ENSG00000000938""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,null,null
"""GTEX-1117F""","""ENSG00000000971""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,"""-0.034286473""",null


### Read specieslm and create bins

In [13]:
specieslm_df = (pl.scan_parquet(specieslm_raw_path).select(['gene_id', 'tissue', 'variant_start', 'variant_end', 'ref', 'alt', 'cse_pos', 'tss', 'strand', 'tss_score', 'pas_score'])
                .rename({'gene_id': 'gene', 'tissue': 'subtissue'})
                .with_columns(gene=pl.col('gene').str.replace(r'([^\.]+)\..+$', "${1}")))
specieslm_df.head().collect()

gene,subtissue,variant_start,variant_end,ref,alt,cse_pos,tss,strand,tss_score,pas_score
str,str,i64,i64,str,str,i64,i64,str,f64,f64
"""ENSG00000173673""","""Adipose - Subcutaneous""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187
"""ENSG00000173673""","""Adipose - Visceral (Omentum)""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187
"""ENSG00000173673""","""Adrenal Gland""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187
"""ENSG00000173673""","""Artery - Aorta""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187
"""ENSG00000173673""","""Artery - Coronary""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187


In [14]:
specieslm_df = specieslm_df.with_columns(tss_distance=(
    pl.when(pl.col('strand') == '+')
    .then(pl.col('variant_start') - pl.col('tss'))
    .otherwise(pl.col('tss') - pl.col('variant_start'))
)).with_columns(absolute_tss_distance=pl.col('tss_distance').abs(),
                absolute_tss_score=pl.col('tss_score').abs())

specieslm_df.select(upstream_tss=pl.col('tss_distance').min(), downstream_tss=pl.col('tss_distance').max()).collect()

upstream_tss,downstream_tss
i64,i64
-2000,2000


In [15]:
specieslm_df = specieslm_df. with_columns(cse_distance=(
    pl.when(pl.col('strand') == '+')
    .then(pl.col('variant_start') - pl.col('cse_pos'))
    .otherwise(pl.col('cse_pos') - pl.col('variant_start'))
)).with_columns(absolute_cse_distance=pl.col('cse_distance').abs(),
                absolute_cse_score=pl.col('pas_score').abs())

specieslm_df.select(upstream_tss=pl.col('cse_distance').min(), downstream_tss=pl.col('cse_distance').max()).collect()

upstream_tss,downstream_tss
i64,i64
-2000,2000


In [16]:
specieslm_df.head().collect()

gene,subtissue,variant_start,variant_end,ref,alt,cse_pos,tss,strand,tss_score,pas_score,tss_distance,absolute_tss_distance,absolute_tss_score,cse_distance,absolute_cse_distance,absolute_cse_score
str,str,i64,i64,str,str,i64,i64,str,f64,f64,i64,i64,f64,i64,i64,f64
"""ENSG00000173673""","""Adipose - Subcutaneous""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187,384,384,0.026847,-985,985,0.044187
"""ENSG00000173673""","""Adipose - Visceral (Omentum)""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187,384,384,0.026847,-985,985,0.044187
"""ENSG00000173673""","""Adrenal Gland""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187,384,384,0.026847,-985,985,0.044187
"""ENSG00000173673""","""Artery - Aorta""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187,384,384,0.026847,-985,985,0.044187
"""ENSG00000173673""","""Artery - Coronary""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""",-0.026847,-0.044187,384,384,0.026847,-985,985,0.044187


In [17]:
upstream=2000
downstream = 2000
df_bac = specieslm_df

# tss_distance in bins
bin_size=250
cuts = list(range(-upstream + bin_size, downstream, bin_size))
cut_labels = [str(x) for x in [-upstream, *cuts]]
tss_distance_labels = {c: f'[{c}, {int(c) + bin_size})' for c in cut_labels}
specieslm_df = (specieslm_df.with_columns(tss_distance_bin=(pl.col('tss_distance').cut(cuts, labels=cut_labels))
                                          .cast(pl.Enum(cut_labels))))

specieslm_df = (specieslm_df.with_columns(tss_distance_bin_label=pl.col("tss_distance_bin").replace_strict(tss_distance_labels),
                                          bin_size=pl.lit(bin_size)))

In [18]:
upstream=2000
downstream = 2000
df_bac = specieslm_df

# cse_distance in bins
bin_size=50
cuts = list(range(-upstream + bin_size, downstream, bin_size))
cut_labels = [str(x) for x in [-upstream, *cuts]]
cse_distance_labels = {c: f'[{c}, {int(c) + bin_size})' for c in cut_labels}
specieslm_df = (specieslm_df.with_columns(cse_distance_bin=(pl.col('cse_distance').cut(cuts, labels=cut_labels))
                                          .cast(pl.Enum(cut_labels))))

specieslm_df = (specieslm_df.with_columns(cse_distance_bin_label=pl.col("cse_distance_bin").replace_strict(cse_distance_labels),
                                          bin_size=pl.lit(50)))

In [19]:
specieslm_df = specieslm_df.with_columns(tss_score=pl.col('tss_score').cast(pl.String), pas_score=pl.col('pas_score').cast(pl.String))

In [20]:
specieslm_df.head().collect()

gene,subtissue,variant_start,variant_end,ref,alt,cse_pos,tss,strand,tss_score,pas_score,tss_distance,absolute_tss_distance,absolute_tss_score,cse_distance,absolute_cse_distance,absolute_cse_score,tss_distance_bin,tss_distance_bin_label,bin_size,cse_distance_bin,cse_distance_bin_label
str,str,i64,i64,str,str,i64,i64,str,str,str,i64,i64,f64,i64,i64,f64,enum,str,i32,enum,str
"""ENSG00000173673""","""Adipose - Subcutaneous""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""","""-0.026847221""","""-0.044186868""",384,384,0.026847,-985,985,0.044187,"""250""","""[250, 500)""",50,"""-1000""","""[-1000, -950)"""
"""ENSG00000173673""","""Adipose - Visceral (Omentum)""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""","""-0.026847221""","""-0.044186868""",384,384,0.026847,-985,985,0.044187,"""250""","""[250, 500)""",50,"""-1000""","""[-1000, -950)"""
"""ENSG00000173673""","""Adrenal Gland""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""","""-0.026847221""","""-0.044186868""",384,384,0.026847,-985,985,0.044187,"""250""","""[250, 500)""",50,"""-1000""","""[-1000, -950)"""
"""ENSG00000173673""","""Artery - Aorta""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""","""-0.026847221""","""-0.044186868""",384,384,0.026847,-985,985,0.044187,"""250""","""[250, 500)""",50,"""-1000""","""[-1000, -950)"""
"""ENSG00000173673""","""Artery - Coronary""",6304622,6304623,"""T""","""C""",6305607,6304238,"""+""","""-0.026847221""","""-0.044186868""",384,384,0.026847,-985,985,0.044187,"""250""","""[250, 500)""",50,"""-1000""","""[-1000, -950)"""


In [21]:
specieslm_benchmark_df = (merged_df.join(specieslm_df, on=['gene', 'subtissue', 'pas_score', 'tss_score'], how='left', join_nulls=True).collect())

In [22]:
specieslm_benchmark_df.head()

individual,gene,subtissue,tissue,hilo_padj,y_pred_old,y_true,y_pred_new,pas_score,tss_score,variant_start,variant_end,ref,alt,cse_pos,tss,strand,tss_distance,absolute_tss_distance,absolute_tss_score,cse_distance,absolute_cse_distance,absolute_cse_score,tss_distance_bin,tss_distance_bin_label,bin_size,cse_distance_bin,cse_distance_bin_label
str,str,str,str,i64,f64,bool,f64,str,str,i64,i64,str,str,i64,i64,str,i64,i64,f64,i64,i64,f64,enum,str,i32,enum,str
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""GTEX-1117F""","""ENSG00000000419""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003511,false,-0.003407,"""-0.03840475""",null,49550123,49550124,"""C""","""T""",49551433,null,"""-""",null,null,null,1310,1310,0.038405,null,null,50,"""1300""","""[1300, 1350)"""
"""GTEX-1117F""","""ENSG00000000457""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003511,false,-0.003407,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""GTEX-1117F""","""ENSG00000000938""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""GTEX-1117F""","""ENSG00000000971""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,"""-0.034286473""",null,196717722,196717723,"""T""","""C""",196716601,null,"""+""",null,null,null,1121,1121,0.034286,null,null,50,"""1100""","""[1100, 1150)"""


In [23]:
specieslm_df.filter((pl.col('gene') == 'ENSG00000000419') & (pl.col('subtissue') == 'Adipose - Subcutaneous') & (pl.col('pas_score') == "-0.03840475")).collect()

gene,subtissue,variant_start,variant_end,ref,alt,cse_pos,tss,strand,tss_score,pas_score,tss_distance,absolute_tss_distance,absolute_tss_score,cse_distance,absolute_cse_distance,absolute_cse_score,tss_distance_bin,tss_distance_bin_label,bin_size,cse_distance_bin,cse_distance_bin_label
str,str,i64,i64,str,str,i64,i64,str,str,str,i64,i64,f64,i64,i64,f64,enum,str,i32,enum,str
"""ENSG00000000419""","""Adipose - Subcutaneous""",49550123,49550124,"""C""","""T""",49551433,null,"""-""",null,"""-0.03840475""",null,null,null,1310,1310,0.038405,null,null,50,"""1300""","""[1300, 1350)"""


In [24]:
specieslm_benchmark_df.write_parquet('/s/project/rep/processed/training_results_v16/specieslm_variants_with_benchmark_250bin.parquet')

## AUPRC

In [9]:
import polars as pl
import gc
from sklearn.metrics import average_precision_score
import plotnine as pn

In [10]:
specieslm_benchmark_df = pl.read_parquet('/s/project/rep/processed/training_results_v16/specieslm_variants_with_benchmark_250bin.parquet')
specieslm_benchmark_df = specieslm_benchmark_df.group_by(['individual', 'gene', 'subtissue', 'pas_score', 'tss_score']).agg(pl.exclude(['gene', 'subtissue', 'pas_score', 'tss_score']).first())

In [11]:
old_tissue_auprc = specieslm_benchmark_df.group_by('tissue').map_groups(lambda group: pl.DataFrame({
    "tissue" : group["tissue"][0],
    "auprc" : average_precision_score(group.select('y_true').to_series(), group.select('y_pred_old').to_series())
}))

In [12]:
old_tissue_auprc.select(pl.col('auprc').median())

auprc
f64
0.058331


In [13]:
new_tissue_auprc = specieslm_benchmark_df.group_by('tissue').map_groups(lambda group: pl.DataFrame({
    "tissue" : group["tissue"][0],
    "auprc" : average_precision_score(group.select('y_true').to_series(), group.select('y_pred_new').to_series())
}))

In [14]:
new_tissue_auprc.select(pl.col('auprc').median())

auprc
f64
0.067075


In [15]:
specieslm_benchmark_df = pl.read_parquet('/s/project/rep/processed/training_results_v16/specieslm_variants_with_benchmark.parquet')
tmp = specieslm_benchmark_df.filter(((pl.col('variant_start') == 47018849) & (pl.col('ref') == 'G') & (pl.col('alt') == 'T') & (pl.col('gene') == 'ENSG00000265681')))

In [16]:
tmp

individual,gene,subtissue,tissue,hilo_padj,y_pred_old,y_true,y_pred_new,pas_score,tss_score,variant_start,variant_end,ref,alt,cse_pos,tss,strand,tss_distance,absolute_tss_distance,absolute_tss_score,cse_distance,absolute_cse_distance,absolute_cse_score,tss_distance_bin,tss_distance_bin_label,bin_size,cse_distance_bin,cse_distance_bin_label
str,str,str,str,i64,f64,bool,f64,str,str,i64,i64,str,str,i64,i64,str,i64,i64,f64,i64,i64,f64,enum,str,i32,enum,str
"""GTEX-YF7O""","""ENSG00000265681""","""Adipose - Subcutaneous""","""Adipose Tissue""",-1,0.231308,true,4.99077,"""-0.050249927""","""-0.100487284""",47018849,47018850,"""G""","""T""",47016875,47017243,"""-""",-1606,1606,0.100487,-1974,1974,0.05025,"""-1650""","""[-1650, -1600)""",50,"""-2000""","""[-2000, -1950)"""
"""GTEX-15CHR""","""ENSG00000265681""","""Cells - Cultured fibroblasts""","""Skin""",-1,0.238502,true,4.723877,"""-0.050249927""","""-0.100487284""",47018849,47018850,"""G""","""T""",47016875,47017243,"""-""",-1606,1606,0.100487,-1974,1974,0.05025,"""-1650""","""[-1650, -1600)""",50,"""-2000""","""[-2000, -1950)"""
"""GTEX-YF7O""","""ENSG00000265681""","""Cells - Cultured fibroblasts""","""Skin""",-1,1.289215,true,4.69782,"""-0.050249927""","""-0.100487284""",47018849,47018850,"""G""","""T""",47016875,47017243,"""-""",-1606,1606,0.100487,-1974,1974,0.05025,"""-1650""","""[-1650, -1600)""",50,"""-2000""","""[-2000, -1950)"""
"""GTEX-YF7O""","""ENSG00000265681""","""Cells - EBV-transformed lympho…","""Blood""",0,1.056078,false,4.69782,"""-0.050249927""","""-0.100487284""",47018849,47018850,"""G""","""T""",47016875,47017243,"""-""",-1606,1606,0.100487,-1974,1974,0.05025,"""-1650""","""[-1650, -1600)""",50,"""-2000""","""[-2000, -1950)"""
"""GTEX-YF7O""","""ENSG00000265681""","""Colon - Sigmoid""","""Colon""",-1,0.231308,true,4.99077,"""-0.050249927""","""-0.100487284""",47018849,47018850,"""G""","""T""",47016875,47017243,"""-""",-1606,1606,0.100487,-1974,1974,0.05025,"""-1650""","""[-1650, -1600)""",50,"""-2000""","""[-2000, -1950)"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""GTEX-YF7O""","""ENSG00000265681""","""Thyroid""","""Thyroid""",-1,0.223519,true,4.987852,"""-0.050249927""","""-0.100487284""",47018849,47018850,"""G""","""T""",47016875,47017243,"""-""",-1606,1606,0.100487,-1974,1974,0.05025,"""-1650""","""[-1650, -1600)""",50,"""-2000""","""[-2000, -1950)"""
"""GTEX-15CHR""","""ENSG00000265681""","""Whole Blood""","""Blood""",-1,0.054891,true,4.217158,"""-0.050249927""","""-0.100487284""",47018849,47018850,"""G""","""T""",47016875,47017243,"""-""",-1606,1606,0.100487,-1974,1974,0.05025,"""-1650""","""[-1650, -1600)""",50,"""-2000""","""[-2000, -1950)"""
"""GTEX-YF7O""","""ENSG00000265681""","""Whole Blood""","""Blood""",-1,0.112803,true,4.003615,"""-0.050249927""","""-0.100487284""",47018849,47018850,"""G""","""T""",47016875,47017243,"""-""",-1606,1606,0.100487,-1974,1974,0.05025,"""-1650""","""[-1650, -1600)""",50,"""-2000""","""[-2000, -1950)"""


In [17]:
tmp.select(pl.col('individual')).unique()

individual
str
"""GTEX-15CHR"""
"""GTEX-YF7O"""


In [18]:
tmp.select(pl.col('tissue')).unique()

tissue
str
"""Adrenal Gland"""
"""Muscle"""
"""Colon"""
"""Esophagus"""
"""Blood Vessel"""
…
"""Pancreas"""
"""Salivary Gland"""
"""Testis"""


In [19]:
specieslm_wo_benchmark_df = (specieslm_benchmark_df
                          .filter(~((pl.col('variant_start') == 47018849) & (pl.col('ref') == 'G') & (pl.col('alt') == 'T') & (pl.col('gene') == 'ENSG00000265681'))))
old_wo_tissue_auprc = specieslm_wo_benchmark_df.group_by('tissue').map_groups(lambda group: pl.DataFrame({
    "tissue" : group["tissue"][0],
    "auprc" : average_precision_score(group.select('y_true').to_series(), group.select('y_pred_old').to_series())
}))
new_wo_tissue_auprc = specieslm_wo_benchmark_df.group_by('tissue').map_groups(lambda group: pl.DataFrame({
    "tissue" : group["tissue"][0],
    "auprc" : average_precision_score(group.select('y_true').to_series(), group.select('y_pred_new').to_series())
}))

In [20]:
print(old_wo_tissue_auprc.select(pl.col('auprc').median()))
print(new_wo_tissue_auprc.select(pl.col('auprc').median()))

shape: (1, 1)
┌──────────┐
│ auprc    │
│ ---      │
│ f64      │
╞══════════╡
│ 0.058558 │
└──────────┘
shape: (1, 1)
┌──────────┐
│ auprc    │
│ ---      │
│ f64      │
╞══════════╡
│ 0.058164 │
└──────────┘


In [21]:
specieslm_benchmark_df = specieslm_benchmark_df.with_columns(cse_distance_bin_label=pl.col('cse_distance_bin_label').fill_null('no_label'),
                                                             tss_distance_bin_label=pl.col('tss_distance_bin_label').fill_null('no_label'),)

In [22]:
specieslm_benchmark_df.head()

individual,gene,subtissue,tissue,hilo_padj,y_pred_old,y_true,y_pred_new,pas_score,tss_score,variant_start,variant_end,ref,alt,cse_pos,tss,strand,tss_distance,absolute_tss_distance,absolute_tss_score,cse_distance,absolute_cse_distance,absolute_cse_score,tss_distance_bin,tss_distance_bin_label,bin_size,cse_distance_bin,cse_distance_bin_label
str,str,str,str,i64,f64,bool,f64,str,str,i64,i64,str,str,i64,i64,str,i64,i64,f64,i64,i64,f64,enum,str,i32,enum,str
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""no_label""",null,null,"""no_label"""
"""GTEX-1117F""","""ENSG00000000419""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003511,false,-0.003407,"""-0.03840475""",null,49550123,49550124,"""C""","""T""",49551433,null,"""-""",null,null,null,1310,1310,0.038405,null,"""no_label""",50,"""1300""","""[1300, 1350)"""
"""GTEX-1117F""","""ENSG00000000457""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.003511,false,-0.003407,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""no_label""",null,null,"""no_label"""
"""GTEX-1117F""","""ENSG00000000938""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""no_label""",null,null,"""no_label"""
"""GTEX-1117F""","""ENSG00000000971""","""Adipose - Subcutaneous""","""Adipose Tissue""",0,-0.00246,false,-0.00251,"""-0.034286473""",null,196717722,196717723,"""T""","""C""",196716601,null,"""+""",null,null,null,1121,1121,0.034286,null,"""no_label""",50,"""1100""","""[1100, 1150)"""


In [23]:
def bin_auprc_new(group_df, is_tss=True):
    bin_column = 'tss_distance_bin_label' if is_tss else 'cse_distance_bin_label'
    bin_name = group_df[bin_column][0]
    tissue = group_df['tissue'][0]
    subset_df = specieslm_benchmark_df.filter((pl.col('tissue') == tissue) & (pl.col(bin_column) != bin_name))
    y_true = pl.concat([subset_df.select('y_true').to_series(), group_df.select('y_true').to_series()])
    y_pred = pl.concat([subset_df.select('y_pred_old').to_series(), group_df.select('y_pred_new').to_series()])
    auprc = average_precision_score(y_true, y_pred)

    return pl.DataFrame({'tissue': tissue,
                         'auprc_new': auprc,
                         'bin_name': bin_name,})

In [24]:
# tmp = specieslm_benchmark_df.filter((pl.col('tissue') == 'Nerve') &
#                                      (pl.col('cse_distance_bin_label') == '[1300, 1350)'))

In [25]:
# bin_auprc_new(tmp, is_tss=False)

In [26]:
old_tissue_auprc = old_tissue_auprc.rename(dict(auprc='auprc_old'))

In [27]:
tss_auprc_bins = specieslm_benchmark_df.group_by('tss_distance_bin_label', 'tissue').map_groups(lambda group: bin_auprc_new(group, is_tss=True))

In [ ]:
cse_auprc_bins = specieslm_benchmark_df.group_by('cse_distance_bin_label', 'tissue').map_groups(lambda group: bin_auprc_new(group, is_tss=False))

In [ ]:
cse_auprc_bins.join(old_tissue_auprc, how='left', on='tissue').write_parquet('cse_auprc_bins_250.parquet')

In [ ]:
tss_auprc_bins.join(old_tissue_auprc, how='left', on='tissue').write_parquet('tss_auprc_bins_250.parquet')

In [ ]:
tss_auprc_bins

In [ ]:
1